### Menu Driven code

In [2]:
import pandas as pd

def load_files():
    input_file = ""
    reference_file = ""
    main_df = pd.read_excel(input_file)
    reference_df = pd.read_csv(reference_file)
    print("\n🔹 FILES LOADED 🔹")
    print(f"Main file → {len(main_df)} rows")
    print(f"Reference file → {len(reference_df)} rows\n")
    return main_df, reference_df

def export_file(df):
    output_file = ""
    df.to_excel(output_file, index=False)
    print(f"\n✅ Cleaned data saved to: {output_file}\n")

def remove_duplicates(df, mobile_column, email_column):
    print("\n🔹 STEP: Remove Duplicates 🔹")
    duplicate_rows = df[df.duplicated(subset=[mobile_column, email_column], keep='first')]
    before = len(df)
    df_no_dup = df.drop_duplicates(subset=[mobile_column, email_column], keep='first')
    removed = before - len(df_no_dup)
    print(f"Duplicates removed (based on {mobile_column}, {email_column}): {removed}")
    if removed > 0:
        print("Sample duplicates removed:")
        print(duplicate_rows[[mobile_column, email_column]].head(), "\n")
    return df_no_dup

def clean_names(df, name_column):
    print("\n🔹 STEP: Clean Incorrect Names 🔹")
    original_names = df[name_column].copy()
    df[name_column] = df[name_column].str.replace(r'^[^a-zA-Z]+', '', regex=True)
    changed_rows = original_names != df[name_column]
    changed_count = changed_rows.sum()
    if changed_count > 0:
        print(f"Rows with cleaned names: {changed_count}")
        name_changes = pd.DataFrame({
            'Original': original_names[changed_rows],
            'Cleaned': df[name_column][changed_rows]
        })
        print("Sample name changes:")
        print(name_changes.head(), "\n")
    cleaned_df = df[~changed_rows]
    print(f"Rows removed due to incorrect names: {changed_count}\n")
    return cleaned_df

def remove_blank_names(df, name_column):
    print("\n🔹 STEP: Remove Blank Names 🔹")
    before = len(df)
    df_no_blank = df[df[name_column].str.strip() != '']
    removed = before - len(df_no_blank)
    print(f"Blank name rows removed: {removed}")
    if removed > 0:
        print("Sample blank names removed:")
        print(df[df[name_column].str.strip() == ''].head(), "\n")
    return df_no_blank

def remove_common_mobiles(df, reference_df, mobile_column, ref_mobile_column):
    print("\n🔹 STEP: Remove Common Mobiles (against reference) 🔹")
    before = len(df)
    filtered_df = df[~df[mobile_column].isin(reference_df[ref_mobile_column])]
    removed = before - len(filtered_df)
    print(f"Common mobiles removed: {removed}")
    if removed > 0:
        print("Sample common mobiles removed:")
        print(df[df[mobile_column].isin(reference_df[ref_mobile_column])][[mobile_column]].head(), "\n")
    return filtered_df

def validate_mobile_numbers(df, mobile_column):
    print("\n🔹 STEP: Validate Mobile Numbers (10-digit only) 🔹")
    before = len(df)
    valid_mobile_pattern = r'^\d{10}$'
    invalid_mobiles = df[~df[mobile_column].astype(str).str.match(valid_mobile_pattern)]
    valid_df = df[df[mobile_column].astype(str).str.match(valid_mobile_pattern)]
    removed = before - len(valid_df)
    print(f"Invalid mobile number rows removed: {removed}")
    if removed > 0:
        print("Sample invalid mobiles removed:")
        print(invalid_mobiles[[mobile_column]].head(), "\n")
    return valid_df

def remove_custom_duplicates(df):
    print("\n🔹 STEP: Remove Custom Duplicates 🔹")
    cols_input = input("Enter the column names to check duplicates (comma-separated): ")
    columns = [col.strip() for col in cols_input.split(',')]
    duplicate_rows = df[df.duplicated(subset=columns, keep='first')]
    before = len(df)
    df_no_dup = df.drop_duplicates(subset=columns, keep='first')
    removed = before - len(df_no_dup)
    print(f"Custom duplicates removed (based on {columns}): {removed}")
    if removed > 0:
        print("Sample duplicates removed:")
        print(duplicate_rows[columns].head(), "\n")
    return df_no_dup

def main_menu():
    print("\n⚙️ SETUP COLUMNS ⚙️")
    mobile_column = input("Enter the Mobile column name (default 'Mobile'): ") or 'Mobile'
    email_column = input("Enter the Email column name (default 'Email'): ") or 'Email'
    name_column = input("Enter the Name column name (default 'Name'): ") or 'Name'
    ref_mobile_column = input("Enter the reference mobile column name (default 'MOBILE'): ") or 'MOBILE'
    
    main_df, reference_df = load_files()
    
    while True:
        print("\n======================")
        print("📋 MAIN MENU 📋")
        print("======================")
        print("1️⃣  Remove duplicates (Mobile + Email)")
        print("2️⃣  Clean incorrect names")
        print("3️⃣  Remove blank names")
        print("4️⃣  Remove common mobiles")
        print("5️⃣  Validate mobile numbers (10-digit only)")
        print("6️⃣  Export cleaned file")
        print("7️⃣  Run ALL steps")
        print("8️⃣  Remove duplicates on custom columns")
        print("0️⃣  Exit")
        print("======================")
        
        choice = input("Enter your choice (0-8): ")
        
        if choice == '1':
            main_df = remove_duplicates(main_df, mobile_column, email_column)
        elif choice == '2':
            main_df = clean_names(main_df, name_column)
        elif choice == '3':
            main_df = remove_blank_names(main_df, name_column)
        elif choice == '4':
            main_df = remove_common_mobiles(main_df, reference_df, mobile_column, ref_mobile_column)
        elif choice == '5':
            main_df = validate_mobile_numbers(main_df, mobile_column)
        elif choice == '6':
            export_file(main_df)
        elif choice == '7':
            main_df = remove_duplicates(main_df, mobile_column, email_column)
            main_df = clean_names(main_df, name_column)
            main_df = remove_blank_names(main_df, name_column)
            main_df = remove_common_mobiles(main_df, reference_df, mobile_column, ref_mobile_column)
            main_df = validate_mobile_numbers(main_df, mobile_column)
            export_file(main_df)
        elif choice == '8':
            main_df = remove_custom_duplicates(main_df)
        elif choice == '0':
            print("\n👋 Exiting program. Goodbye! 👋\n")
            break
        else:
            print("\n⚠️ Invalid choice. Please enter a number from the menu.\n")

# Run the menu
main_menu()


⚙️ SETUP COLUMNS ⚙️


Enter the Mobile column name (default 'Mobile'):  Mobile
Enter the Email column name (default 'Email'):  Email
Enter the Name column name (default 'Name'):  Name
Enter the reference mobile column name (default 'MOBILE'):  MOBILE



🔹 FILES LOADED 🔹
Main file → 20680 rows
Reference file → 186013 rows


📋 MAIN MENU 📋
1️⃣  Remove duplicates (Mobile + Email)
2️⃣  Clean incorrect names
3️⃣  Remove blank names
4️⃣  Remove common mobiles
5️⃣  Validate mobile numbers (10-digit only)
6️⃣  Export cleaned file
7️⃣  Run ALL steps
8️⃣  Remove duplicates on custom columns
0️⃣  Exit


Enter your choice (0-8):  7



🔹 STEP: Remove Duplicates 🔹
Duplicates removed (based on Mobile, Email): 1
Sample duplicates removed:
     Mobile               Email
2  21345215  awdawdsh@gmail.com 


🔹 STEP: Clean Incorrect Names 🔹
Rows with cleaned names: 29
Sample name changes:
    Original Cleaned
0    123aMan    aMan
41       NaN     NaN
47       NaN     NaN
62       NaN     NaN
977      NaN     NaN 

Rows removed due to incorrect names: 29


🔹 STEP: Remove Blank Names 🔹
Blank name rows removed: 0

🔹 STEP: Remove Common Mobiles (against reference) 🔹
Common mobiles removed: 0

🔹 STEP: Validate Mobile Numbers (10-digit only) 🔹
Invalid mobile number rows removed: 21
Sample invalid mobiles removed:
                       Mobile
1                    21345215
7837                912340004
10955  9718864664, 9625791534
10967  9711375383/ 9818067255
10989   9815230220/9915376006 


✅ Cleaned data saved to: C:/Users/sinha/Desktop/Inter SU/Data Cleaning/output_final.xlsx


📋 MAIN MENU 📋
1️⃣  Remove duplicates (Mobile + E

...


🔹 STEP: Remove Custom Duplicates 🔹


Enter the column names to check duplicates (comma-separated):  Mobile


Custom duplicates removed (based on ['Mobile']): 75
Sample duplicates removed:
          Mobile
1680  9958203331
1973  9599076501
3535  8700728210
3974  9599076501
5578  9958203331 


📋 MAIN MENU 📋
1️⃣  Remove duplicates (Mobile + Email)
2️⃣  Clean incorrect names
3️⃣  Remove blank names
4️⃣  Remove common mobiles
5️⃣  Validate mobile numbers (10-digit only)
6️⃣  Export cleaned file
7️⃣  Run ALL steps
8️⃣  Remove duplicates on custom columns
0️⃣  Exit


Enter your choice (0-8):  0



👋 Exiting program. Goodbye! 👋



### Enter location of files

In [1]:
# Path to the input file (the file you want to clean)
input_file_path = r""

# Path to the reference file (for filtering common entries)
reference_file_path = r""

# Path where cleaned file will be saved
output_file_path = r""

### Remove Duplicates (keeps first occurance)

In [2]:
import pandas as pd
main_df = pd.read_excel(input_file_path)
reference_df = pd.read_csv(reference_file_path)

original_row_count = len(main_df)

# Remove duplicates based on 'Mobile' and 'Email' columns
main_df_no_duplicates = main_df.drop_duplicates(subset=['Mobile', 'Email'], keep='first')

# Find out how many rows were removed
duplicates_removed_count = original_row_count - len(main_df_no_duplicates)

print(f"Duplicates removed: {duplicates_removed_count}")

# (Optional) See which rows were removed
removed_rows = main_df[main_df.duplicated(subset=['Mobile', 'Email'], keep='first')]
removed_rows.head()  # Display first few removed rows

# Save the cleaned dataframe to Excel
filtered_df.to_excel(output_file_path, index=False)

print(f"Cleaned data saved to: {output_file_path}")

NameError: name 'main_df' is not defined

### Remove Rows with Incorrect Names (Non-Alphabetic Start)

In [ ]:
# Make a copy of the original 'Name' column
original_names = main_df_no_duplicates['Name'].copy()

# Clean 'Name' column by removing any non-letter characters at the start
main_df_no_duplicates['Name'] = main_df_no_duplicates['Name'].str.replace(r'^[^a-zA-Z]+', '', regex=True)

# Identify which rows changed
name_changed_rows = original_names != main_df_no_duplicates['Name']
name_changes_count = name_changed_rows.sum()

# Remove rows where name was changed (if that’s the rule)
cleaned_df = main_df_no_duplicates[~name_changed_rows]

print(f"Rows removed due to incorrect names: {name_changes_count}")

# Save the cleaned dataframe to Excel
filtered_df.to_excel(output_file_path, index=False)

print(f"Cleaned data saved to: {output_file_path}")

### Remove Blank or Space-Only Names

In [ ]:
# Remove rows where 'Name' is just blank or spaces
final_df = cleaned_df[cleaned_df['Name'].str.strip() != '']

print(f"Remaining rows after removing blanks: {len(final_df)}")

### Remove Common Entries Based on 'Mobile'

In [ ]:
# Filter out rows from final_df where 'Mobile' exists in reference_df['MOBILE']
filtered_df = final_df[~final_df['Mobile'].isin(reference_df['MOBILE'])]

print(f"Rows after removing common mobiles: {len(filtered_df)}")

### Exporting

In [ ]:
# Save the cleaned dataframe to Excel
filtered_df.to_excel(output_file_path, index=False)

print(f"Cleaned data saved to: {output_file_path}")